In [7]:
sf=1

# GENERATE TPCH PARQUET FILES

In [8]:
%pip install duckdb==0.8.1
%pip install numpy
%pip  install matplotlib
%pip show duckdb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Name: duckdb
Version: 0.8.1
Summary: DuckDB embedded database
Home-page: https://www.duckdb.org
Author: 
Author-email: 
License: MIT
Location: /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages
Requires: 
Required-by: duckdb-engine, tpch-duckdb-bench
Note: you may need to restart the kernel to use updated packages.


In [9]:
import duckdb
import time
import pandas as pd
import random

# basic SQL API

# connect to an in-memory temporary database
#conn = duckdb.connect()
conn = duckdb.connect(f"./rw/RW.DUCKDB")
# if you want, you can create a cursor() like described in PEP 249 but its fully redundant
cursor = conn.cursor()

# run arbitrary SQL commands
conn.execute("CREATE TABLE IF NOT EXISTS test_table  (i INTEGER, j STRING)")

df = pd.DataFrame(columns=['dur'])


# single insert
test_type='single insert'
start=time.time()
conn.execute("INSERT INTO test_table VALUES (1, 'one')")
stop = time.time()
duration = stop-start
print(duration)

row = {'dur': duration,'type':test_type }
df = pd.concat([df,pd.DataFrame(row, index=[test_type])], axis=0, ignore_index=True)

# Bulk Insert Data
test_type='bulk insert'
start=time.time()

for num in range(0,99999):
    # n=int(num)
    n=random.randint(0,99999)

    conn.execute("INSERT INTO test_table VALUES (?, ?)", [n, 'one'])

stop = time.time()

duration = stop-start
print(duration)

row = {'dur': duration,'type':test_type }
df = pd.concat([df,pd.DataFrame(row, index=[test_type])], axis=0, ignore_index=True)


# single select
test_type='single search'
start=time.time()
conn.execute("select * from test_table where i =99998")
stop = time.time()
duration = stop-start
print(duration)

row = {'dur': duration,'type':test_type }
df = pd.concat([df,pd.DataFrame(row, index=[test_type])], axis=0, ignore_index=True)

# bulk select
test_type='bulk select'
start=time.time()
for num in range(0,99999):
    n=int(num)
    conn.execute("select * from test_table where i = ?",[n])

stop = time.time()
duration = stop-start
print(duration)

row = {'dur': duration,'type':test_type }
df = pd.concat([df,pd.DataFrame(row, index=[test_type])], axis=0, ignore_index=True)
conn.close()
print(df)



0.0009589195251464844
9.122267961502075
0.00017261505126953125
7.298788785934448
        dur           type
0  0.000959  single insert
1  9.122268    bulk insert
2  0.000173  single search
3  7.298789    bulk select
